# 🚀 LSTM Crypto Price Prediction - Google Colab Version

**Instructions:**
1. Upload your sequence data to Google Drive
2. Run this notebook on Colab with GPU enabled
3. Download trained models when done

**Enable GPU:** Runtime → Change runtime type → GPU

## Step 1: Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Check GPU
import tensorflow as tf
print(f"TensorFlow: {tf.__version__}")
print(f"GPU: {tf.config.list_physical_devices('GPU')}")

## Step 2: Upload Sequence Data

**Before running the next cell:**
1. Create folder in Google Drive: `MyDrive/crypto_project/`
2. Upload `sequences.zip` from your local `Milestone_1/data/sequences/` folder

**To create sequences.zip locally, run:**
```bash
cd /Users/ayushgupta/Desktop/ML-Driven-Web-Platform-for-Cryptocurrency-Price-Forecasting_November_Batch-5_2025/Milestone_1/data
zip -r sequences.zip sequences/
```

In [ ]:
import os
import zipfile

# Paths
DRIVE_PATH = "/content/drive/MyDrive/crypto_project"
WORK_DIR = "/content/crypto_training"
SEQ_ZIP = f"{DRIVE_PATH}/sequences.zip"

# Create working directory
os.makedirs(WORK_DIR, exist_ok=True)
os.makedirs(f"{WORK_DIR}/models", exist_ok=True)

# Extract sequences
if os.path.exists(SEQ_ZIP):
    print("Extracting sequences.zip...")
    with zipfile.ZipFile(SEQ_ZIP, 'r') as z:
        z.extractall(WORK_DIR)
    print("✅ Extracted!")
else:
    print(f"❌ ERROR: Upload sequences.zip to {DRIVE_PATH}")

# Verify extraction
!ls -la {WORK_DIR}/sequences/

## Step 3: Configuration

**Change these values to train different models:**

In [ ]:
# ======= CONFIGURATION =======
COIN_NAME = "bitcoin"  # Options: bitcoin, ethereum, solana, cardano, binancecoin
HORIZON = "1h"         # Options: 1h, 24h

# Paths
SEQ_PATH = f"{WORK_DIR}/sequences/{COIN_NAME}/{HORIZON}/"
MODEL_SAVE_PATH = f"{WORK_DIR}/models/{COIN_NAME}/{HORIZON}/"
os.makedirs(MODEL_SAVE_PATH, exist_ok=True)

print(f"🎯 Training: {COIN_NAME} - {HORIZON}")
print(f"📁 Sequences: {SEQ_PATH}")
print(f"💾 Models: {MODEL_SAVE_PATH}")

## Step 4: Load Data

In [ ]:
import numpy as np

X_train = np.load(SEQ_PATH + "X_train.npy")
y_train = np.load(SEQ_PATH + "y_train.npy")
X_val = np.load(SEQ_PATH + "X_val.npy")
y_val = np.load(SEQ_PATH + "y_val.npy")
X_test = np.load(SEQ_PATH + "X_test.npy")
y_test = np.load(SEQ_PATH + "y_test.npy")

print("=" * 50)
print("DATA LOADED")
print("=" * 50)
print(f"X_train: {X_train.shape}")
print(f"y_train: {y_train.shape} → range [{y_train.min():.4f}, {y_train.max():.4f}]")
print(f"X_val:   {X_val.shape}")
print(f"X_test:  {X_test.shape}")

## Step 5: Build Improved Model

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2

def build_model(input_shape, l2_reg=0.001):
    model = Sequential([
        LSTM(32, return_sequences=True, input_shape=input_shape,
             kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(0.2),
        
        LSTM(16, kernel_regularizer=l2(l2_reg), recurrent_regularizer=l2(l2_reg)),
        BatchNormalization(),
        Dropout(0.2),
        
        Dense(8, activation="relu", kernel_regularizer=l2(l2_reg)),
        Dense(1, activation="sigmoid")  # Output [0, 1]
    ])
    return model

# Build
input_shape = (X_train.shape[1], X_train.shape[2])
model = build_model(input_shape)
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss="mse", metrics=["mae"])
model.summary()

## Step 6: Train Model

In [ ]:
# Callbacks
callbacks = [
    ModelCheckpoint(f"{MODEL_SAVE_PATH}/best_lstm_{COIN_NAME}_{HORIZON}.keras",
                    monitor="val_loss", save_best_only=True, verbose=1),
    EarlyStopping(monitor="val_loss", patience=20, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=5, min_lr=1e-6, verbose=1)
]

print("=" * 50)
print(f"🚀 TRAINING: {COIN_NAME} - {HORIZON}")
print("=" * 50)

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
    verbose=1
)

## Step 7: Evaluate & Save

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, r2_score

# Training curves
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(history.history['loss'], label='Train')
axes[0].plot(history.history['val_loss'], label='Val')
axes[0].set_title('Loss'); axes[0].legend()
axes[1].plot(history.history['mae'], label='Train')
axes[1].plot(history.history['val_mae'], label='Val')
axes[1].set_title('MAE'); axes[1].legend()
plt.tight_layout()
plt.show()

# Evaluate
preds = model.predict(X_test, verbose=0).flatten()
mae = mean_absolute_error(y_test, preds)
r2 = r2_score(y_test, preds)

print("\n" + "=" * 50)
print("📊 TEST RESULTS")
print("=" * 50)
print(f"MAE: {mae:.4f}")
print(f"R²:  {r2:.4f}")
print(f"Prediction range: [{preds.min():.4f}, {preds.max():.4f}]")

if r2 > 0:
    print("✅ Model is learning!")
else:
    print("⚠️ Negative R² - needs improvement")

In [ ]:
# Plot predictions
plt.figure(figsize=(14, 5))
plt.plot(y_test[:500], label='Actual', alpha=0.8)
plt.plot(preds[:500], label='Predicted', alpha=0.8)
plt.title(f'{COIN_NAME} {HORIZON} - Predictions vs Actual')
plt.legend()
plt.show()

In [ ]:
# Save final model
final_path = f"{MODEL_SAVE_PATH}/final_lstm_{COIN_NAME}_{HORIZON}.keras"
model.save(final_path)
print(f"\n✅ Model saved: {final_path}")

## Step 8: Copy Models to Google Drive

In [ ]:
import shutil

# Create models folder in Drive
DRIVE_MODELS = f"{DRIVE_PATH}/trained_models/{COIN_NAME}/{HORIZON}"
os.makedirs(DRIVE_MODELS, exist_ok=True)

# Copy models
for f in os.listdir(MODEL_SAVE_PATH):
    if f.endswith('.keras'):
        src = f"{MODEL_SAVE_PATH}/{f}"
        dst = f"{DRIVE_MODELS}/{f}"
        shutil.copy(src, dst)
        print(f"✅ Copied: {f}")

print(f"\n📁 Models saved to: {DRIVE_MODELS}")

---

## 🔄 Train Next Model

**To train the next coin/horizon:**
1. Go back to **Step 3** (Configuration cell)
2. Change `COIN_NAME` and/or `HORIZON`
3. Run all cells from Step 3 onwards

**Training Matrix:**
```
bitcoin     1h ✓  24h ✓
ethereum    1h ✓  24h ✓
solana      1h ✓  24h ✓
cardano     1h ✓  24h ✓
binancecoin 1h ✓  24h ✓
```

## Step 9: Download All Models (After Training All)

Run this **after training all 10 models** to zip them for download:

In [ ]:
# Zip all trained models
!cd {DRIVE_PATH} && zip -r trained_models.zip trained_models/
print(f"\n📦 Download from: {DRIVE_PATH}/trained_models.zip")
print("\nThen extract to your local Milestone_2/models/ folder")